In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet18
from torch.ao.quantization import get_default_qconfig, prepare, convert, QConfigMapping
import os
import time

# 1. 하이퍼파라미터 및 설정
BATCH_SIZE = 64
NUM_EPOCHS = 5 # 양자화 전에 모델을 어느 정도 학습시키는 것이 좋습니다.
CALIBRATION_BATCHES = 100 # Calibration에 사용할 배치 수 (MNIST 학습 데이터셋에서)
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. ResNet 모델 수정 (MNIST 1채널 입력용)
class ResNet18_MNIST(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18_MNIST, self).__init__()
        # 사전 학습된 ResNet18을 불러오지만, weights는 로드하지 않습니다.
        # ImageNet_V1 Weights를 로드하려면 conv1을 다시 초기화해야 합니다.
        original_resnet = resnet18(weights=None)

        # conv1 레이어를 1채널 입력에 맞게 수정
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = original_resnet.bn1
        self.relu = original_resnet.relu
        self.maxpool = original_resnet.maxpool

        self.layer1 = original_resnet.layer1
        self.layer2 = original_resnet.layer2
        self.layer3 = original_resnet.layer3
        self.layer4 = original_resnet.layer4

        self.avgpool = original_resnet.avgpool
        self.fc = nn.Linear(original_resnet.fc.in_features, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# 3. 데이터 로더 준비 (MNIST)
transform = transforms.Compose([
    transforms.ToTensor(), # 이미지를 텐서로 변환
    transforms.Normalize((0.1307,), (0.3081,)) # MNIST 평균, 표준편차로 정규화
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
calibration_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


# 4. 모델 학습 (양자화 전에 모델을 학습시킵니다)
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    model.to(DEVICE)
    print(f"\n모델 학습 시작 ({num_epochs} 에폭)...")
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99: # 100 배치마다 출력
                print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}")
                running_loss = 0.0
    print("모델 학습 완료.")

# 5. 모델 평가 함수
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')
    return accuracy

# 6. 모델 생성 및 초기 학습
model_fp32 = ResNet18_MNIST(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_fp32.parameters(), lr=LEARNING_RATE)

# GPU가 있다면 모델을 GPU로 이동
model_fp32.to(DEVICE)

# 학습 실행
train_model(model_fp32, train_loader, criterion, optimizer, NUM_EPOCHS)

print("\n--- FP32 모델 평가 ---")
fp32_accuracy = evaluate_model(model_fp32, test_loader)
torch.save(model_fp32.state_dict(), "resnet18_mnist_fp32.pth")
fp32_size = os.path.getsize("resnet18_mnist_fp32.pth") / (1024 * 1024)
print(f"FP32 모델 크기: {fp32_size:.2f} MB")

# 7. Static Quantization 준비
# 모델을 CPU로 옮깁니다. PyTorch의 PTQ는 현재 CPU에서 주로 지원됩니다.
model_fp32.to("cpu")
model_fp32.eval() # 평가 모드 설정

# 레이어 퓨징 (성능 향상)
# ResNet의 각 Residual Block 내부에 Conv-BN-ReLU 패턴이 있습니다.
# torchvision.models._utils.fuse_model 함수를 사용하거나 수동으로 퓨징할 수 있습니다.
# 여기서는 간단하게 첫 번째 컨볼루션 레이어만 퓨징합니다.
# 실제 ResNet의 모든 Conv-BN-ReLU를 퓨징하려면 더 복잡한 로직이 필요합니다.
# https://pytorch.org/docs/stable/quantization.html#module-fusion
fused_model = torch.ao.quantization.fuse_modules(model_fp32,
                                               [['conv1', 'bn1', 'relu']],
                                               inplace=False)

# QConfig 설정 (CPU 최적화된 fbgemm 백엔드)
qconfig = get_default_qconfig("fbgemm")
qconfig_mapping = QConfigMapping().set_global(qconfig) # PyTorch 1.13+

# 모델 준비 (양자화 가능한 모듈을 대체하고 Observer 삽입)
#prepared_model = prepare(fused_model, qconfig_mapping, inplace=False)
prepared_model = prepare(fused_model, qconfig_mapping)



# 8. Calibration (보정)
print("\n--- Static Quantization Calibration 시작 ---")
# Calibration loader를 통해 모델에 데이터를 흘려보내 활성화 분포를 측정합니다.
with torch.inference_mode():
    for batch_idx, (inputs, labels) in enumerate(calibration_loader):
        if batch_idx >= CALIBRATION_BATCHES:
            break
        prepared_model(inputs)
        if batch_idx % 20 == 0:
            print(f"  Calibration batch: {batch_idx}/{CALIBRATION_BATCHES}")
print("Calibration 완료.")


# 9. 양자화 적용 및 변환
quantized_model_static = convert(prepared_model, inplace=False)
quantized_model_static.eval()

# 10. 양자화된 모델 평가
print("\n--- Quantized Static 모델 평가 ---")
# 양자화된 모델도 CPU에서 평가합니다.
quantized_accuracy = evaluate_model(quantized_model_static, test_loader)

# 양자화된 모델 저장 (전체 모델 객체 저장)
torch.save(quantized_model_static, "resnet18_mnist_quantized_static.pth")
quantized_size = os.path.getsize("resnet18_mnist_quantized_static.pth") / (1024 * 1024)
print(f"Quantized Static 모델 크기: {quantized_size:.2f} MB")

print("\n--- 최종 결과 비교 ---")
print(f"FP32 모델 정확도: {fp32_accuracy:.2f}% (크기: {fp32_size:.2f} MB)")
print(f"Static Quantized 모델 정확도: {quantized_accuracy:.2f}% (크기: {quantized_size:.2f} MB)")


모델 학습 시작 (5 에폭)...
Epoch [1/5], Batch [100/938], Loss: 0.3509
Epoch [1/5], Batch [200/938], Loss: 0.1738
Epoch [1/5], Batch [300/938], Loss: 0.1296
Epoch [1/5], Batch [400/938], Loss: 0.1133
Epoch [1/5], Batch [500/938], Loss: 0.1113
Epoch [1/5], Batch [600/938], Loss: 0.0841
Epoch [1/5], Batch [700/938], Loss: 0.0895
Epoch [1/5], Batch [800/938], Loss: 0.0664
Epoch [1/5], Batch [900/938], Loss: 0.0725
Epoch [2/5], Batch [100/938], Loss: 0.0689
Epoch [2/5], Batch [200/938], Loss: 0.0623
Epoch [2/5], Batch [300/938], Loss: 0.0605
Epoch [2/5], Batch [400/938], Loss: 0.0690
Epoch [2/5], Batch [500/938], Loss: 0.0629
Epoch [2/5], Batch [600/938], Loss: 0.0615
Epoch [2/5], Batch [700/938], Loss: 0.0672
Epoch [2/5], Batch [800/938], Loss: 0.0469
Epoch [2/5], Batch [900/938], Loss: 0.0571
Epoch [3/5], Batch [100/938], Loss: 0.0405
Epoch [3/5], Batch [200/938], Loss: 0.0498
Epoch [3/5], Batch [300/938], Loss: 0.0368
Epoch [3/5], Batch [400/938], Loss: 0.0371
Epoch [3/5], Batch [500/938], Loss

C:\Users\JH\anaconda\envs\torch_env\lib\site-packages\torch\ao\quantization\quantize.py:388: UserWarning: None of the submodule got qconfig applied. Make sure you passed correct configuration through `qconfig_dict` or by assigning the `.qconfig` attribute directly on submodules
  warnings.warn(


  Calibration batch: 0/100
  Calibration batch: 20/100
  Calibration batch: 40/100
  Calibration batch: 60/100
  Calibration batch: 80/100
Calibration 완료.

--- Quantized Static 모델 평가 ---


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet18
from torch.ao.quantization import get_default_qconfig, prepare, convert, QConfigMapping
import os
import time

# 1. 하이퍼파라미터 및 설정
BATCH_SIZE = 64
NUM_EPOCHS = 5 # 양자화 전에 모델을 어느 정도 학습시키는 것이 좋습니다.
CALIBRATION_BATCHES = 100 # Calibration에 사용할 배치 수 (MNIST 학습 데이터셋에서)
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. ResNet 모델 수정 (MNIST 1채널 입력용)
class ResNet18_MNIST(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18_MNIST, self).__init__()
        # 사전 학습된 ResNet18을 불러오지만, weights는 로드하지 않습니다.
        # ImageNet_V1 Weights를 로드하려면 conv1을 다시 초기화해야 합니다.
        original_resnet = resnet18(weights=None)

        # conv1 레이어를 1채널 입력에 맞게 수정
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = original_resnet.bn1
        self.relu = original_resnet.relu
        self.maxpool = original_resnet.maxpool

        self.layer1 = original_resnet.layer1
        self.layer2 = original_resnet.layer2
        self.layer3 = original_resnet.layer3
        self.layer4 = original_resnet.layer4

        self.avgpool = original_resnet.avgpool
        self.fc = nn.Linear(original_resnet.fc.in_features, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# 3. 데이터 로더 준비 (MNIST)
transform = transforms.Compose([
    transforms.ToTensor(), # 이미지를 텐서로 변환
    transforms.Normalize((0.1307,), (0.3081,)) # MNIST 평균, 표준편차로 정규화
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
calibration_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


# 4. 모델 학습 (양자화 전에 모델을 학습시킵니다)
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    # model.to(DEVICE) # 학습 함수 내에서 DEVICE로 이동합니다.
    print(f"\n모델 학습 시작 ({num_epochs} 에폭)...")
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99: # 100 배치마다 출력
                print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}")
                running_loss = 0.0
    print("모델 학습 완료.")

#---

### 5. 모델 평가 함수 (수정됨)

def evaluate_model(model, test_loader, target_device=None): # target_device 인자 추가
    model.eval()
    correct = 0
    total = 0

    if target_device is None:
        # 모델의 첫 번째 파라미터가 있는 디바이스를 가져옵니다.
        # ResNet18_MNIST는 항상 파라미터를 가지므로 안전합니다.
        model_device = next(model.parameters()).device
    else:
        model_device = target_device

    # 모델을 평가할 디바이스로 옮겨줍니다.
    model.to(model_device)

    with torch.no_grad():
        for inputs, labels in test_loader:
            # 입력 데이터를 모델이 있는 디바이스로 옮깁니다.
            inputs, labels = inputs.to(model_device), labels.to(model_device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')
    return accuracy

In [13]:
# 6. 모델 생성 및 초기 학습
model_fp32 = ResNet18_MNIST(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_fp32.parameters(), lr=LEARNING_RATE)

# GPU가 있다면 모델을 GPU로 이동
# 이 줄이 중요합니다! model_fp32 = model_fp32.to(DEVICE) 로 수정합니다.
model_fp32 = model_fp32.to(DEVICE) 

# 학습 실행
train_model(model_fp32, train_loader, criterion, optimizer, NUM_EPOCHS)

print("\n--- FP32 모델 평가 ---")
# FP32 모델은 학습이 끝난 후 현재 DEVICE (GPU)에 있으므로, 해당 DEVICE에서 평가
fp32_accuracy = evaluate_model(model_fp32, test_loader, target_device=DEVICE)
torch.save(model_fp32.state_dict(), "resnet18_mnist_fp32.pth")
fp32_size = os.path.getsize("resnet18_mnist_fp32.pth") / (1024 * 1024)
print(f"FP32 모델 크기: {fp32_size:.2f} MB")

# 7. Static Quantization 준비
# 모델을 CPU로 옮깁니다. PyTorch의 PTQ는 현재 CPU에서 주로 지원됩니다.
model_fp32.to("cpu")
model_fp32.eval() # 평가 모드 설정

# 레이어 퓨징 (성능 향상)
fused_model = torch.ao.quantization.fuse_modules(model_fp32,
                                               [['conv1', 'bn1', 'relu']],
                                               inplace=False)

# QConfig 설정 (CPU 최적화된 fbgemm 백엔드)
qconfig = get_default_qconfig("fbgemm")
qconfig_mapping = QConfigMapping().set_global(qconfig) # PyTorch 1.13+

# 모델 준비 (양자화 가능한 모듈을 대체하고 Observer 삽입)
prepared_model = prepare(fused_model, qconfig_mapping) # 'inplace=False' 인자 제거


# 8. Calibration (보정)
print("\n--- Static Quantization Calibration 시작 ---")
# Calibration loader를 통해 모델에 데이터를 흘려보내 활성화 분포를 측정합니다.
with torch.inference_mode():
    # Calibration 시에도 모델이 CPU에 있으므로, 입력 데이터를 CPU로 옮겨줍니다.
    for batch_idx, (inputs, labels) in enumerate(calibration_loader):
        if batch_idx >= CALIBRATION_BATCHES:
            break
        inputs = inputs.to("cpu") # Calibration 데이터는 CPU로!
        prepared_model(inputs)
        if batch_idx % 20 == 0:
            print(f"  Calibration batch: {batch_idx}/{CALIBRATION_BATCHES}")
print("Calibration 완료.")


# 9. 양자화 적용 및 변환
quantized_model_static = convert(prepared_model, inplace=False)
quantized_model_static.eval()

# 10. 양자화된 모델 평가 (수정됨)
print("\n--- Quantized Static 모델 평가 ---")
# 양자화된 모델은 CPU에 있으므로, CPU에서 평가하도록 명시합니다.
quantized_accuracy = evaluate_model(quantized_model_static, test_loader, target_device=torch.device("cpu"))

# 양자화된 모델 저장 (전체 모델 객체 저장)
torch.save(quantized_model_static, "resnet18_mnist_quantized_static.pth")
quantized_size = os.path.getsize("resnet18_mnist_quantized_static.pth") / (1024 * 1024)
print(f"Quantized Static 모델 크기: {quantized_size:.2f} MB")

print("\n--- 최종 결과 비교 ---")
print(f"FP32 모델 정확도: {fp32_accuracy:.2f}% (크기: {fp32_size:.2f} MB)")
print(f"Static Quantized 모델 정확도: {quantized_accuracy:.2f}% (크기: {quantized_size:.2f} MB)")


모델 학습 시작 (5 에폭)...
Epoch [1/5], Batch [100/938], Loss: 0.3462
Epoch [1/5], Batch [200/938], Loss: 0.1681
Epoch [1/5], Batch [300/938], Loss: 0.1192
Epoch [1/5], Batch [400/938], Loss: 0.1210
Epoch [1/5], Batch [500/938], Loss: 0.1058
Epoch [1/5], Batch [600/938], Loss: 0.0961
Epoch [1/5], Batch [700/938], Loss: 0.0918
Epoch [1/5], Batch [800/938], Loss: 0.0804
Epoch [1/5], Batch [900/938], Loss: 0.0753
Epoch [2/5], Batch [100/938], Loss: 0.0660
Epoch [2/5], Batch [200/938], Loss: 0.0577
Epoch [2/5], Batch [300/938], Loss: 0.0756
Epoch [2/5], Batch [400/938], Loss: 0.0576
Epoch [2/5], Batch [500/938], Loss: 0.0707
Epoch [2/5], Batch [600/938], Loss: 0.0524
Epoch [2/5], Batch [700/938], Loss: 0.0619
Epoch [2/5], Batch [800/938], Loss: 0.0667
Epoch [2/5], Batch [900/938], Loss: 0.0488
Epoch [3/5], Batch [100/938], Loss: 0.0495
Epoch [3/5], Batch [200/938], Loss: 0.0423
Epoch [3/5], Batch [300/938], Loss: 0.0441
Epoch [3/5], Batch [400/938], Loss: 0.0421
Epoch [3/5], Batch [500/938], Loss

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet18
# FX Graph 기반 양자화를 위한 모듈 임포트
from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx, fuse_fx
from torch.ao.quantization import get_default_qconfig_mapping # QConfigMapping을 가져오는 더 간결한 방법
import os
import time

# 1. 하이퍼파라미터 및 설정
BATCH_SIZE = 64
NUM_EPOCHS = 5 # 양자화 전에 모델을 어느 정도 학습시키는 것이 좋습니다.
CALIBRATION_BATCHES = 100 # Calibration에 사용할 배치 수 (MNIST 학습 데이터셋에서)
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. ResNet 모델 수정 (MNIST 1채널 입력용)
class ResNet18_MNIST(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18_MNIST, self).__init__()
        # 사전 학습된 ResNet18을 불러오지만, weights는 로드하지 않습니다.
        original_resnet = resnet18(weights=None)

        # conv1 레이어를 1채널 입력에 맞게 수정
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = original_resnet.bn1
        self.relu = original_resnet.relu
        self.maxpool = original_resnet.maxpool

        self.layer1 = original_resnet.layer1
        self.layer2 = original_resnet.layer2
        self.layer3 = original_resnet.layer3
        self.layer4 = original_resnet.layer4

        self.avgpool = original_resnet.avgpool
        self.fc = nn.Linear(original_resnet.fc.in_features, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# 3. 데이터 로더 준비 (MNIST)
transform = transforms.Compose([
    transforms.ToTensor(), # 이미지를 텐서로 변환
    transforms.Normalize((0.1307,), (0.3081,)) # MNIST 평균, 표준편차로 정규화
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
calibration_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


# 4. 모델 학습 (양자화 전에 모델을 학습시킵니다)
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    # model.to(DEVICE) # 모델 이동은 호출하는 쪽에서 명시적으로 처리
    print(f"\n모델 학습 시작 ({num_epochs} 에폭)...")
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99: # 100 배치마다 출력
                print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}")
                running_loss = 0.0
    print("모델 학습 완료.")

# 5. 모델 평가 함수 (이전과 동일)
def evaluate_model(model, test_loader, target_device=None):
    model.eval()
    correct = 0
    total = 0

    if target_device is None:
        model_device = next(model.parameters()).device
    else:
        model_device = target_device

    model.to(model_device) # 모델을 평가할 디바이스로 옮겨줍니다.

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(model_device), labels.to(model_device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')
    return accuracy
'''
# 6. 모델 생성 및 초기 학습
model_fp32 = ResNet18_MNIST(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_fp32.parameters(), lr=LEARNING_RATE)

# GPU가 있다면 모델을 GPU로 이동
model_fp32 = model_fp32.to(DEVICE)

# 학습 실행
train_model(model_fp32, train_loader, criterion, optimizer, NUM_EPOCHS)

print("\n--- FP32 모델 평가 ---")
fp32_accuracy = evaluate_model(model_fp32, test_loader, target_device=DEVICE)
torch.save(model_fp32.state_dict(), "resnet18_mnist_fp32.pth")
fp32_size = os.path.getsize("resnet18_mnist_fp32.pth") / (1024 * 1024)
print(f"FP32 모델 크기: {fp32_size:.2f} MB")
'''
# 6. 모델 생성 및 초기 학습
# model_fp32 = ResNet18_MNIST(num_classes=10) # 이 부분 변경
# model_fp32 = resnet18(weights=None) # torchvision의 기본 ResNet18 로드
model_fp32 = resnet18(weights=None) # torchvision.models.resnet18에서 로드

# conv1 레이어 수정 (1채널 입력에 맞게)
model_fp32.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
# fc 레이어 수정 (MNIST 10개 클래스에 맞게)
model_fp32.fc = nn.Linear(model_fp32.fc.in_features, 10)

# 양자화 친화적으로 만들기 위한 Stubs 삽입 (FX 양자화에는 불필요하지만 일반 양자화에 필요)
# FX 양자화에서는 내부적으로 처리되지만, 명시적으로 추가하는 것은 나쁘지 않습니다.
# model_fp32 = nn.Sequential(
#     QuantStub(), # 입력 양자화를 위한 QuantStub
#     model_fp32,
#     DeQuantStub() # 출력 역양자화를 위한 DeQuantStub
# )


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_fp32.parameters(), lr=LEARNING_RATE)

model_fp32 = model_fp32.to(DEVICE) # 모델을 GPU로 이동

train_model(model_fp32, train_loader, criterion, optimizer, NUM_EPOCHS)

print("\n--- FP32 모델 평가 ---")
fp32_accuracy = evaluate_model(model_fp32, test_loader, target_device=DEVICE)
torch.save(model_fp32.state_dict(), "resnet18_mnist_fp32.pth")
fp32_size = os.path.getsize("resnet18_mnist_fp32.pth") / (1024 * 1024)
print(f"FP32 모델 크기: {fp32_size:.2f} MB")

# 7. Static Quantization 준비 (FX Graph 기반으로 수정)
# 모델을 CPU로 옮깁니다. FX 양자화도 현재 CPU에서 주로 지원됩니다.
model_fp32.to("cpu")
model_fp32.eval() # 평가 모드 설정

# 7-1. FX 그래프 기반 모델 퓨징
# torchvision.models.quantization.utils.fuse_model을 사용하거나,
# fuse_fx를 사용하는 경우에도 모델 구조가 표준에 가까워야 함.
# 여기서는 torchvision의 양자화 친화적 ResNet을 사용하므로 fuse_fx가 더 잘 작동할 가능성이 높음.
# 모델을 직접 fuse_fx에 전달하면 됩니다.
fused_model = fuse_fx(model_fp32)
print("FX Graph 퓨징 완료.")

# 7-2. QConfig 설정
qconfig_mapping = get_default_qconfig_mapping("fbgemm")

# 7-3. 모델 준비 (FX 그래프 기반)
example_input = torch.randn(1, 1, 28, 28) # MNIST 입력 크기 (배치1, 채널1, 높이28, 너비28)
prepared_model = prepare_fx(fused_model, qconfig_mapping, example_inputs=(example_input,))
print("FX Graph prepare 완료.")

# 8. Calibration (보정) - 이전과 동일
print("\n--- Static Quantization Calibration 시작 ---")
with torch.inference_mode():
    for batch_idx, (inputs, labels) in enumerate(calibration_loader):
        if batch_idx >= CALIBRATION_BATCHES:
            break
        inputs = inputs.to("cpu") # Calibration 데이터는 CPU로!
        prepared_model(inputs)
        if batch_idx % 20 == 0:
            print(f"  Calibration batch: {batch_idx}/{CALIBRATION_BATCHES}")
print("Calibration 완료.")


# 9. 양자화 적용 및 변환 (FX Graph 기반으로 수정)
quantized_model_static = convert_fx(prepared_model)
quantized_model_static.eval()
print("FX Graph convert 완료.")

# 10. 양자화된 모델 평가 (이전과 동일)
print("\n--- Quantized Static 모델 평가 ---")
quantized_accuracy = evaluate_model(quantized_model_static, test_loader, target_device=torch.device("cpu"))

# 양자화된 모델 저장 (전체 모델 객체 저장)
torch.save(quantized_model_static, "resnet18_mnist_quantized_static_fx.pth") # 파일명 변경
quantized_size = os.path.getsize("resnet18_mnist_quantized_static_fx.pth") / (1024 * 1024)
print(f"Quantized Static (FX) 모델 크기: {quantized_size:.2f} MB")

print("\n--- 최종 결과 비교 ---")
print(f"FP32 모델 정확도: {fp32_accuracy:.2f}% (크기: {fp32_size:.2f} MB)")
print(f"Static Quantized (FX) 모델 정확도: {quantized_accuracy:.2f}% (크기: {quantized_size:.2f} MB)")


모델 학습 시작 (5 에폭)...
Epoch [1/5], Batch [100/938], Loss: 0.3756
Epoch [1/5], Batch [200/938], Loss: 0.1647
Epoch [1/5], Batch [300/938], Loss: 0.1524
Epoch [1/5], Batch [400/938], Loss: 0.1093
Epoch [1/5], Batch [500/938], Loss: 0.1072
Epoch [1/5], Batch [600/938], Loss: 0.0920
Epoch [1/5], Batch [700/938], Loss: 0.0789
Epoch [1/5], Batch [800/938], Loss: 0.0892
Epoch [1/5], Batch [900/938], Loss: 0.0991
Epoch [2/5], Batch [100/938], Loss: 0.0457
Epoch [2/5], Batch [200/938], Loss: 0.0668
Epoch [2/5], Batch [300/938], Loss: 0.0616
Epoch [2/5], Batch [400/938], Loss: 0.0651
Epoch [2/5], Batch [500/938], Loss: 0.0617
Epoch [2/5], Batch [600/938], Loss: 0.0526
Epoch [2/5], Batch [700/938], Loss: 0.0585
Epoch [2/5], Batch [800/938], Loss: 0.0664
Epoch [2/5], Batch [900/938], Loss: 0.0589
Epoch [3/5], Batch [100/938], Loss: 0.0413
Epoch [3/5], Batch [200/938], Loss: 0.0415
Epoch [3/5], Batch [300/938], Loss: 0.0409
Epoch [3/5], Batch [400/938], Loss: 0.0430
Epoch [3/5], Batch [500/938], Loss

C:\Users\JH\anaconda\envs\torch_env\lib\site-packages\torch\ao\quantization\observer.py:244: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


FX Graph prepare 완료.

--- Static Quantization Calibration 시작 ---
  Calibration batch: 0/100
  Calibration batch: 20/100
  Calibration batch: 40/100
  Calibration batch: 60/100
  Calibration batch: 80/100
Calibration 완료.


KeyError: 'layer1.0.conv1.1'

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet18 # torchvision의 기본 resnet18 임포트
# FX Graph 기반 양자화를 위한 모듈 임포트
from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx, fuse_fx
from torch.ao.quantization import get_default_qconfig_mapping # QConfigMapping을 가져오는 간결한 방법
import os
import time

# 1. 하이퍼파라미터 및 설정
BATCH_SIZE = 64
NUM_EPOCHS = 5 # 양자화 전에 모델을 어느 정도 학습시키는 것이 좋습니다.
CALIBRATION_BATCHES = 100 # Calibration에 사용할 배치 수 (MNIST 학습 데이터셋에서)
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. ResNet 모델 정의 (이제 torchvision의 기본 resnet18을 사용하고, conv1과 fc만 수정)
# 이전의 ResNet18_MNIST 클래스는 이제 필요 없습니다.

# 3. 데이터 로더 준비 (MNIST)
transform = transforms.Compose([
    transforms.ToTensor(), # 이미지를 텐서로 변환
    transforms.Normalize((0.1307,), (0.3081,)) # MNIST 평균, 표준편차로 정규화
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
calibration_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


# 4. 모델 학습 함수
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    print(f"\n모델 학습 시작 ({num_epochs} 에폭)...")
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99: # 100 배치마다 출력
                print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}")
                running_loss = 0.0
    print("모델 학습 완료.")

# 5. 모델 평가 함수
def evaluate_model(model, test_loader, target_device=None):
    model.eval()
    correct = 0
    total = 0

    if target_device is None:
        # 모델의 첫 번째 파라미터가 있는 디바이스를 가져옵니다.
        model_device = next(model.parameters()).device
    else:
        model_device = target_device

    model.to(model_device) # 모델을 평가할 디바이스로 옮겨줍니다.

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(model_device), labels.to(model_device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')
    return accuracy

# --- 메인 실행 로직 ---

# 6. 모델 생성 및 초기 학습 (torchvision의 resnet18 사용)
model_fp32 = resnet18(weights=None) # 사전 학습된 가중치 없이 로드

# conv1 레이어 수정 (1채널 입력에 맞게)
# ResNet은 원래 3채널을 받으므로, MNIST 흑백 이미지(1채널)에 맞춰 수정
model_fp32.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
# fc 레이어 수정 (MNIST 10개 클래스에 맞게)
model_fp32.fc = nn.Linear(model_fp32.fc.in_features, 10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_fp32.parameters(), lr=LEARNING_RATE)

# GPU가 있다면 모델을 GPU로 이동 (학습 전에 필수)
model_fp32 = model_fp32.to(DEVICE)

# 학습 실행
train_model(model_fp32, train_loader, criterion, optimizer, NUM_EPOCHS)

print("\n--- FP32 모델 평가 ---")
fp32_accuracy = evaluate_model(model_fp32, test_loader, target_device=DEVICE)
torch.save(model_fp32.state_dict(), "resnet18_mnist_fp32.pth")
fp32_size = os.path.getsize("resnet18_mnist_fp32.pth") / (1024 * 1024)
print(f"FP32 모델 크기: {fp32_size:.2f} MB")



### Static Quantization (FX Graph 기반)

# 7. Static Quantization 준비 (FX Graph 기반)
# 모델을 CPU로 옮깁니다. FX 양자화는 주로 CPU에서 이루어집니다.
model_fp32.to("cpu")
model_fp32.eval() # 평가 모드 설정

# 7-1. FX 그래프 기반 모델 퓨징
# 모든 Conv-BN-ReLU, Conv-ReLU 등의 패턴을 자동으로 퓨징합니다.
fused_model = fuse_fx(model_fp32)
print("FX Graph 퓨징 완료.")

# 7-2. QConfig 설정 (CPU 최적화된 fbgemm 백엔드)
qconfig_mapping = get_default_qconfig_mapping("fbgemm")

# 7-3. 모델 준비 (FX 그래프 기반)
# example_inputs가 필요합니다. 이는 모델의 계산 그래프를 트레이싱할 때 사용됩니다.
# MNIST 입력 크기: (배치1, 채널1, 높이28, 너비28)
example_input = torch.randn(1, 1, 28, 28)
# inplace=False를 명시적으로 추가
prepared_model = prepare_fx(fused_model, qconfig_mapping, example_inputs=(example_input,))
print("FX Graph prepare 완료.")

# 8. Calibration (보정)
print("\n--- Static Quantization Calibration 시작 ---")
with torch.inference_mode():
    for batch_idx, (inputs, labels) in enumerate(calibration_loader):
        if batch_idx >= CALIBRATION_BATCHES:
            break
        inputs = inputs.to("cpu") # Calibration 데이터는 CPU로!
        prepared_model(inputs)
        if batch_idx % 20 == 0:
            print(f"  Calibration batch: {batch_idx}/{CALIBRATION_BATCHES}")
print("Calibration 완료.")


# 9. 양자화 적용 및 변환 (FX Graph 기반)
# inplace=False를 명시적으로 추가
quantized_model_static = convert_fx(prepared_model)
quantized_model_static.eval()
print("FX Graph convert 완료.")

# 10. 양자화된 모델 평가
print("\n--- Quantized Static 모델 평가 ---")
# 양자화된 모델은 CPU에 있으므로, CPU에서 평가하도록 명시합니다.
quantized_accuracy = evaluate_model(quantized_model_static, test_loader, target_device=torch.device("cpu"))

# 양자화된 모델 저장 (전체 모델 객체 저장)
torch.save(quantized_model_static, "resnet18_mnist_quantized_static_fx.pth") # 파일명 변경
quantized_size = os.path.getsize("resnet18_mnist_quantized_static_fx.pth") / (1024 * 1024)
print(f"Quantized Static (FX) 모델 크기: {quantized_size:.2f} MB")

print("\n--- 최종 결과 비교 ---")
print(f"FP32 모델 정확도: {fp32_accuracy:.2f}% (크기: {fp32_size:.2f} MB)")
print(f"Static Quantized (FX) 모델 정확도: {quantized_accuracy:.2f}% (크기: {quantized_size:.2f} MB)")


모델 학습 시작 (5 에폭)...
Epoch [1/5], Batch [100/938], Loss: 0.3427
Epoch [1/5], Batch [200/938], Loss: 0.1869
Epoch [1/5], Batch [300/938], Loss: 0.1222
Epoch [1/5], Batch [400/938], Loss: 0.1132
Epoch [1/5], Batch [500/938], Loss: 0.1099
Epoch [1/5], Batch [600/938], Loss: 0.1094
Epoch [1/5], Batch [700/938], Loss: 0.0952
Epoch [1/5], Batch [800/938], Loss: 0.0876
Epoch [1/5], Batch [900/938], Loss: 0.0817
Epoch [2/5], Batch [100/938], Loss: 0.0549
Epoch [2/5], Batch [200/938], Loss: 0.0543
Epoch [2/5], Batch [300/938], Loss: 0.0600
Epoch [2/5], Batch [400/938], Loss: 0.0717
Epoch [2/5], Batch [500/938], Loss: 0.0525
Epoch [2/5], Batch [600/938], Loss: 0.0608
Epoch [2/5], Batch [700/938], Loss: 0.0527
Epoch [2/5], Batch [800/938], Loss: 0.0618
Epoch [2/5], Batch [900/938], Loss: 0.0591
Epoch [3/5], Batch [100/938], Loss: 0.0363
Epoch [3/5], Batch [200/938], Loss: 0.0546
Epoch [3/5], Batch [300/938], Loss: 0.0416
Epoch [3/5], Batch [400/938], Loss: 0.0377
Epoch [3/5], Batch [500/938], Loss

C:\Users\JH\anaconda\envs\torch_env\lib\site-packages\torch\ao\quantization\observer.py:244: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


FX Graph prepare 완료.

--- Static Quantization Calibration 시작 ---
  Calibration batch: 0/100
  Calibration batch: 20/100
  Calibration batch: 40/100
  Calibration batch: 60/100
  Calibration batch: 80/100
Calibration 완료.


KeyError: 'layer1.0.conv1.1'

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet18
from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx, fuse_fx
from torch.ao.quantization import get_default_qconfig_mapping
# Add QuantStub and DeQuantStub imports for explicit control
from torch.ao.quantization import QuantStub, DeQuantStub
import os
import time

# ... (rest of the initial setup, hyperparameters, data loaders, train_model, evaluate_model) ...

# --- Main execution logic ---

# 6. 모델 생성 및 초기 학습
model_fp32 = resnet18(weights=None)

model_fp32.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model_fp32.fc = nn.Linear(model_fp32.fc.in_features, 10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_fp32.parameters(), lr=LEARNING_RATE)

model_fp32 = model_fp32.to(DEVICE)
train_model(model_fp32, train_loader, criterion, optimizer, NUM_EPOCHS)

print("\n--- FP32 모델 평가 ---")
fp32_accuracy = evaluate_model(model_fp32, test_loader, target_device=DEVICE)
torch.save(model_fp32.state_dict(), "resnet18_mnist_fp32.pth")
fp32_size = os.path.getsize("resnet18_mnist_fp32.pth") / (1024 * 1024)
print(f"FP32 모델 크기: {fp32_size:.2f} MB")

# --- Static Quantization (FX Graph 기반) ---

# 7. Static Quantization 준비
model_fp32.to("cpu")
model_fp32.eval()

# Explicitly insert QuantStub and DeQuantStub
# Wrap the model within Sequential with QuantStub and DeQuantStub
# This makes the input/output quantization explicit for the tracer
model_to_quantize = nn.Sequential(
    QuantStub(),
    model_fp32,
    DeQuantStub()
)

# Now apply fuse_fx on this wrapped model
fused_model = fuse_fx(model_to_quantize) # Apply fuse_fx on the wrapped model
print("FX Graph 퓨징 완료.")

qconfig_mapping = get_default_qconfig_mapping("fbgemm")

example_input = torch.randn(1, 1, 28, 28)
# Prepare_fx on the fused model with example inputs
prepared_model = prepare_fx(fused_model, qconfig_mapping, example_inputs=(example_input,))
print("FX Graph prepare 완료.")

# 8. Calibration
print("\n--- Static Quantization Calibration 시작 ---")
with torch.inference_mode():
    for batch_idx, (inputs, labels) in enumerate(calibration_loader):
        if batch_idx >= CALIBRATION_BATCHES:
            break
        inputs = inputs.to("cpu")
        prepared_model(inputs)
        if batch_idx % 20 == 0:
            print(f"  Calibration batch: {batch_idx}/{CALIBRATION_BATCHES}")
print("Calibration 완료.")

# 9. 양자화 적용 및 변환
# Removed 'inplace=False' as it's not accepted by convert_fx
quantized_model_static = convert_fx(prepared_model)
quantized_model_static.eval()
print("FX Graph convert 완료.")

# ... (rest of evaluation and saving) ...


모델 학습 시작 (5 에폭)...
Epoch [1/5], Batch [100/938], Loss: 0.3681
Epoch [1/5], Batch [200/938], Loss: 0.1724
Epoch [1/5], Batch [300/938], Loss: 0.1211
Epoch [1/5], Batch [400/938], Loss: 0.1261
Epoch [1/5], Batch [500/938], Loss: 0.1104
Epoch [1/5], Batch [600/938], Loss: 0.0953
Epoch [1/5], Batch [700/938], Loss: 0.0849
Epoch [1/5], Batch [800/938], Loss: 0.0874
Epoch [1/5], Batch [900/938], Loss: 0.0819
Epoch [2/5], Batch [100/938], Loss: 0.0721
Epoch [2/5], Batch [200/938], Loss: 0.0570
Epoch [2/5], Batch [300/938], Loss: 0.0464
Epoch [2/5], Batch [400/938], Loss: 0.0660
Epoch [2/5], Batch [500/938], Loss: 0.0629
Epoch [2/5], Batch [600/938], Loss: 0.0541
Epoch [2/5], Batch [700/938], Loss: 0.0558
Epoch [2/5], Batch [800/938], Loss: 0.0592
Epoch [2/5], Batch [900/938], Loss: 0.0594
Epoch [3/5], Batch [100/938], Loss: 0.0450
Epoch [3/5], Batch [200/938], Loss: 0.0474
Epoch [3/5], Batch [300/938], Loss: 0.0434
Epoch [3/5], Batch [400/938], Loss: 0.0430
Epoch [3/5], Batch [500/938], Loss

C:\Users\JH\anaconda\envs\torch_env\lib\site-packages\torch\ao\quantization\observer.py:244: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


FX Graph prepare 완료.

--- Static Quantization Calibration 시작 ---
  Calibration batch: 0/100
  Calibration batch: 20/100
  Calibration batch: 40/100
  Calibration batch: 60/100
  Calibration batch: 80/100
Calibration 완료.


KeyError: '1.layer1.0.conv1.1'

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet18
from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx, fuse_fx
# QConfig와 Observer를 직접 임포트하여 제어
from torch.ao.quantization import get_default_qconfig_mapping, QuantStub, DeQuantStub, default_per_channel_weight_observer, default_observer
import os
import time

# 1. 하이퍼파라미터 및 설정
BATCH_SIZE = 64
NUM_EPOCHS = 5
CALIBRATION_BATCHES = 100
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 3. 데이터 로더 준비 (MNIST)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
calibration_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# 4. 모델 학습 함수
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    print(f"\n모델 학습 시작 ({num_epochs} 에폭)...")
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}")
                running_loss = 0.0
    print("모델 학습 완료.")

# 5. 모델 평가 함수
def evaluate_model(model, test_loader, target_device=None):
    model.eval()
    correct = 0
    total = 0

    if target_device is None:
        model_device = next(model.parameters()).device
    else:
        model_device = target_device

    model.to(model_device)

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(model_device), labels.to(model_device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')
    return accuracy

# --- 메인 실행 로직 ---

# 6. 모델 생성 및 초기 학습 (torchvision의 resnet18 사용)
model_fp32 = resnet18(weights=None)

model_fp32.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model_fp32.fc = nn.Linear(model_fp32.fc.in_features, 10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_fp32.parameters(), lr=LEARNING_RATE)

model_fp32 = model_fp32.to(DEVICE)
train_model(model_fp32, train_loader, criterion, optimizer, NUM_EPOCHS)

print("\n--- FP32 모델 평가 ---")
fp32_accuracy = evaluate_model(model_fp32, test_loader, target_device=DEVICE)
torch.save(model_fp32.state_dict(), "resnet18_mnist_fp32.pth")
fp32_size = os.path.getsize("resnet18_mnist_fp32.pth") / (1024 * 1024)
print(f"FP32 모델 크기: {fp32_size:.2f} MB")

# --- Static Quantization (FX Graph 기반) ---

# 7. Static Quantization 준비
model_fp32.to("cpu")
model_fp32.eval()

# Explicitly insert QuantStub and DeQuantStub
# Wrap the model within Sequential with QuantStub and DeQuantStub
model_to_quantize = nn.Sequential(
    QuantStub(),
    model_fp32,
    DeQuantStub()
)

# 7-1. FX 그래프 기반 모델 퓨징
fused_model = fuse_fx(model_to_quantize)
print("FX Graph 퓨징 완료.")

# 7-2. QConfig 설정 (직접 정의하여 Static Quantization 유도)
# fbgemm 백엔드에서 정적 양자화를 위한 QConfig
# default_observer: 활성화 값 (Activation)의 min/max를 측정
# default_per_channel_weight_observer: 가중치(Weight)의 per-channel min/max를 측정
'''qconfig = torch.ao.quantization.QConfig(
    activation=default_observer.with_args(qscheme=torch.per_tensor_symmetric, dtype=torch.qint8),
    weight=default_per_channel_weight_observer.with_args(qscheme=torch.per_channel_symmetric, dtype=torch.qint8)
)
'''

qconfig = torch.ao.quantization.QConfig(
    activation=default_observer.with_args(qscheme=torch.per_tensor_symmetric, dtype=torch.qint8), # 활성화를 8비트 정수로
    weight=default_per_channel_weight_observer.with_args(qscheme=torch.per_channel_symmetric, dtype=torch.qint8) # 가중치를 8비트 정수로
)
# QConfigMapping을 직접 생성하고 글로벌 QConfig 설정
qconfig_mapping = get_default_qconfig_mapping()
qconfig_mapping.set_global(qconfig)

print(f"QConfig Mapping: {qconfig_mapping}")


# 7-3. 모델 준비 (FX 그래프 기반)
example_input = torch.randn(1, 1, 28, 28)
prepared_model = prepare_fx(fused_model, qconfig_mapping, example_inputs=(example_input,))
print("FX Graph prepare 완료.")

# 8. Calibration
print("\n--- Static Quantization Calibration 시작 ---")
with torch.inference_mode():
    for batch_idx, (inputs, labels) in enumerate(calibration_loader):
        if batch_idx >= CALIBRATION_BATCHES:
            break
        inputs = inputs.to("cpu")
        prepared_model(inputs)
        if batch_idx % 20 == 0:
            print(f"  Calibration batch: {batch_idx}/{CALIBRATION_BATCHES}")
print("Calibration 완료.")

# 9. 양자화 적용 및 변환
quantized_model_static = convert_fx(prepared_model)
quantized_model_static.eval()
print("FX Graph convert 완료.")

# 10. 양자화된 모델 평가
print("\n--- Quantized Static 모델 평가 ---")
quantized_accuracy = evaluate_model(quantized_model_static, test_loader, target_device=torch.device("cpu"))

torch.save(quantized_model_static, "resnet18_mnist_quantized_static_fx.pth")
quantized_size = os.path.getsize("resnet18_mnist_quantized_static_fx.pth") / (1024 * 1024)
print(f"Quantized Static (FX) 모델 크기: {quantized_size:.2f} MB")

print("\n--- 최종 결과 비교 ---")
print(f"FP32 모델 정확도: {fp32_accuracy:.2f}% (크기: {fp32_size:.2f} MB)")
print(f"Static Quantized (FX) 모델 정확도: {quantized_accuracy:.2f}% (크기: {quantized_size:.2f} MB)")


모델 학습 시작 (5 에폭)...
Epoch [1/5], Batch [100/938], Loss: 0.3495
Epoch [1/5], Batch [200/938], Loss: 0.1813
Epoch [1/5], Batch [300/938], Loss: 0.1525
Epoch [1/5], Batch [400/938], Loss: 0.1040
Epoch [1/5], Batch [500/938], Loss: 0.0855
Epoch [1/5], Batch [600/938], Loss: 0.1114
Epoch [1/5], Batch [700/938], Loss: 0.0730
Epoch [1/5], Batch [800/938], Loss: 0.0933
Epoch [1/5], Batch [900/938], Loss: 0.0741
Epoch [2/5], Batch [100/938], Loss: 0.0656
Epoch [2/5], Batch [200/938], Loss: 0.0621
Epoch [2/5], Batch [300/938], Loss: 0.0572
Epoch [2/5], Batch [400/938], Loss: 0.0707
Epoch [2/5], Batch [500/938], Loss: 0.0584
Epoch [2/5], Batch [600/938], Loss: 0.0604
Epoch [2/5], Batch [700/938], Loss: 0.0545
Epoch [2/5], Batch [800/938], Loss: 0.0552
Epoch [2/5], Batch [900/938], Loss: 0.0576
Epoch [3/5], Batch [100/938], Loss: 0.0520
Epoch [3/5], Batch [200/938], Loss: 0.0345
Epoch [3/5], Batch [300/938], Loss: 0.0380
Epoch [3/5], Batch [400/938], Loss: 0.0398
Epoch [3/5], Batch [500/938], Loss

In [33]:
print("\n--- 양자화된 모델 가중치 타입 확인 ---")
for name, module in quantized_model_static.named_modules():
    # Conv2d 또는 Linear 레이어인 경우에만 확인
    if isinstance(module, (nn.Conv2d, nn.Linear)):
        if hasattr(module, 'weight') and module.weight is not None:
            print(f"모듈: {name}, 가중치 Dtype: {module.weight.dtype}")


--- 양자화된 모델 가중치 타입 확인 ---
모듈: 1.conv1.0, 가중치 Dtype: torch.float32
모듈: 1.layer1.0.conv1.0, 가중치 Dtype: torch.float32
모듈: 1.layer1.0.conv2, 가중치 Dtype: torch.float32
모듈: 1.layer1.1.conv1.0, 가중치 Dtype: torch.float32
모듈: 1.layer1.1.conv2, 가중치 Dtype: torch.float32
모듈: 1.layer2.0.conv1.0, 가중치 Dtype: torch.float32
모듈: 1.layer2.0.conv2, 가중치 Dtype: torch.float32
모듈: 1.layer2.0.downsample.0, 가중치 Dtype: torch.float32
모듈: 1.layer2.1.conv1.0, 가중치 Dtype: torch.float32
모듈: 1.layer2.1.conv2, 가중치 Dtype: torch.float32
모듈: 1.layer3.0.conv1.0, 가중치 Dtype: torch.float32
모듈: 1.layer3.0.conv2, 가중치 Dtype: torch.float32
모듈: 1.layer3.0.downsample.0, 가중치 Dtype: torch.float32
모듈: 1.layer3.1.conv1.0, 가중치 Dtype: torch.float32
모듈: 1.layer3.1.conv2, 가중치 Dtype: torch.float32
모듈: 1.layer4.0.conv1.0, 가중치 Dtype: torch.float32
모듈: 1.layer4.0.conv2, 가중치 Dtype: torch.float32
모듈: 1.layer4.0.downsample.0, 가중치 Dtype: torch.float32
모듈: 1.layer4.1.conv1.0, 가중치 Dtype: torch.float32
모듈: 1.layer4.1.conv2, 가중치 Dtype: torch.float32
모듈:

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
# torchvision.models.quantization에서 resnet18_quantized를 임포트
from torchvision.models.quantization import resnet18 as resnet18_q
from torch.ao.quantization import get_default_qconfig_mapping, QuantStub, DeQuantStub
import os
import time

# 1. 하이퍼파라미터 및 설정
BATCH_SIZE = 64
NUM_EPOCHS = 5
CALIBRATION_BATCHES = 100
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 3. 데이터 로더 준비 (MNIST)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
calibration_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# 4. 모델 학습 함수
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    print(f"\n모델 학습 시작 ({num_epochs} 에폭)...")
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}")
                running_loss = 0.0
    print("모델 학습 완료.")

# 5. 모델 평가 함수
def evaluate_model(model, test_loader, target_device=None):
    model.eval()
    correct = 0
    total = 0

    if target_device is None:
        model_device = next(model.parameters()).device
    else:
        model_device = target_device

    model.to(model_device)

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(model_device), labels.to(model_device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')
    return accuracy

# --- 메인 실행 로직 ---

# 6. 모델 생성 및 초기 학습 (torchvision.models.quantization의 resnet18_quantized 사용)
# quantize=True로 설정하면 양자화 준비된 모델을 로드합니다.
# 이 모델은 이미 fuse_modules와 QuantStub/DeQuantStub이 적용되어 있습니다.
model_fp32 = resnet18_q(weights=None, quantize=False) # 일단 FP32 버전으로 로드

# conv1 레이어 수정 (1채널 입력에 맞게)
# 이 모델은 이미 양자화 친화적으로 퓨징되어 있으므로,
# conv1 수정 후 수동 퓨징이나 stub 추가는 필요 없습니다.
model_fp32.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model_fp32.fc = nn.Linear(model_fp32.fc.in_features, 10)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_fp32.parameters(), lr=LEARNING_RATE)

model_fp32 = model_fp32.to(DEVICE)
train_model(model_fp32, train_loader, criterion, optimizer, NUM_EPOCHS)

print("\n--- FP32 모델 평가 ---")
fp32_accuracy = evaluate_model(model_fp32, test_loader, target_device=DEVICE)
torch.save(model_fp32.state_dict(), "resnet18_mnist_fp32.pth")
fp32_size = os.path.getsize("resnet18_mnist_fp32.pth") / (1024 * 1024)
print(f"FP32 모델 크기: {fp32_size:.2f} MB")

# --- Static Quantization (torchvision.models.quantization 방식 사용) ---

# 7. Static Quantization 준비
model_fp32.to("cpu") # 모델을 CPU로 옮깁니다.
model_fp32.eval()    # 평가 모드 설정

# torchvision의 양자화 친화적 모델은 QConfig와 옵저버를 이미 내장하고 있습니다.
# 우리는 모델을 로드할 때 이미 (weights=None, quantize=False)로 로드했으므로,
# 이제 qconfig_mapping을 설정하고 prepare_fx를 호출합니다.

# QConfig 설정 (CPU 최적화된 fbgemm 백엔드, 8비트 정수 명시)
qconfig = torch.ao.quantization.QConfig(
    activation=torch.ao.quantization.default_observer.with_args(qscheme=torch.per_tensor_symmetric, dtype=torch.qint8),
    weight=torch.ao.quantization.default_per_channel_weight_observer.with_args(qscheme=torch.per_channel_symmetric, dtype=torch.qint8)
)
qconfig_mapping = get_default_qconfig_mapping()
qconfig_mapping.set_global(qconfig)

# prepare_fx는 자동으로 torchvision 모델의 내부 모듈을 퓨징합니다.
# example_inputs는 prepare_fx에 여전히 필요합니다.
example_input = torch.randn(1, 1, 28, 28)
prepared_model = torch.ao.quantization.quantize_fx.prepare_fx(
    model_fp32, # 이제 model_fp32 자체를 넘겨줍니다. (Sequential로 감싸지 않습니다)
    qconfig_mapping,
    example_inputs=(example_input,),
    # inplace=False는 prepare_fx에서는 허용될 수 있지만, convert_fx에서는 불필요합니다.
    # 여기서는 명시적으로 True/False 대신 기본값(False)을 사용하겠습니다.
)
print("FX Graph prepare 완료.")

# 8. Calibration
print("\n--- Static Quantization Calibration 시작 ---")
with torch.inference_mode():
    for batch_idx, (inputs, labels) in enumerate(calibration_loader):
        if batch_idx >= CALIBRATION_BATCHES:
            break
        inputs = inputs.to("cpu")
        prepared_model(inputs)
        if batch_idx % 20 == 0:
            print(f"  Calibration batch: {batch_idx}/{CALIBRATION_BATCHES}")
print("Calibration 완료.")

# 9. 양자화 적용 및 변환
# convert_fx는 'inplace' 인수를 받지 않습니다.
quantized_model_static = torch.ao.quantization.quantize_fx.convert_fx(prepared_model)
quantized_model_static.eval()
print("FX Graph convert 완료.")

# 10. 양자화된 모델 평가
print("\n--- Quantized Static 모델 평가 ---")
quantized_accuracy = evaluate_model(quantized_model_static, test_loader, target_device=torch.device("cpu"))

# 양자화된 모델 저장 (전체 모델 객체 저장)
torch.save(quantized_model_static, "resnet18_mnist_quantized_static_fx.pth")
quantized_size = os.path.getsize("resnet18_mnist_quantized_static_fx.pth") / (1024 * 1024)
print(f"Quantized Static (FX) 모델 크기: {quantized_size:.2f} MB")

print("\n--- 최종 결과 비교 ---")
print(f"FP32 모델 정확도: {fp32_accuracy:.2f}% (크기: {fp32_size:.2f} MB)")
print(f"Static Quantized (FX) 모델 정확도: {quantized_accuracy:.2f}% (크기: {quantized_size:.2f} MB)")


모델 학습 시작 (5 에폭)...
Epoch [1/5], Batch [100/938], Loss: 0.3552
Epoch [1/5], Batch [200/938], Loss: 0.1721
Epoch [1/5], Batch [300/938], Loss: 0.1421
Epoch [1/5], Batch [400/938], Loss: 0.1134
Epoch [1/5], Batch [500/938], Loss: 0.1022
Epoch [1/5], Batch [600/938], Loss: 0.0996
Epoch [1/5], Batch [700/938], Loss: 0.0804
Epoch [1/5], Batch [800/938], Loss: 0.0923
Epoch [1/5], Batch [900/938], Loss: 0.0831
Epoch [2/5], Batch [100/938], Loss: 0.0600
Epoch [2/5], Batch [200/938], Loss: 0.0682
Epoch [2/5], Batch [300/938], Loss: 0.0645
Epoch [2/5], Batch [400/938], Loss: 0.0646
Epoch [2/5], Batch [500/938], Loss: 0.0557
Epoch [2/5], Batch [600/938], Loss: 0.0633
Epoch [2/5], Batch [700/938], Loss: 0.0584
Epoch [2/5], Batch [800/938], Loss: 0.0504
Epoch [2/5], Batch [900/938], Loss: 0.0542
Epoch [3/5], Batch [100/938], Loss: 0.0523
Epoch [3/5], Batch [200/938], Loss: 0.0402
Epoch [3/5], Batch [300/938], Loss: 0.0640
Epoch [3/5], Batch [400/938], Loss: 0.0421
Epoch [3/5], Batch [500/938], Loss

In [37]:
print("\n--- 양자화된 모델 가중치 타입 확인 ---")
for name, module in quantized_model_static.named_modules():
    # Conv2d 또는 Linear 레이어인 경우에만 확인
    if isinstance(module, (nn.Conv2d, nn.Linear)):
        if hasattr(module, 'weight') and module.weight is not None:
            print(f"모듈: {name}, 가중치 Dtype: {module.weight.dtype}")


--- 양자화된 모델 가중치 타입 확인 ---
모듈: conv1.0, 가중치 Dtype: torch.float32
모듈: layer1.0.conv1.0, 가중치 Dtype: torch.float32
모듈: layer1.0.conv2, 가중치 Dtype: torch.float32
모듈: layer1.1.conv1.0, 가중치 Dtype: torch.float32
모듈: layer1.1.conv2, 가중치 Dtype: torch.float32
모듈: layer2.0.conv1.0, 가중치 Dtype: torch.float32
모듈: layer2.0.conv2, 가중치 Dtype: torch.float32
모듈: layer2.0.downsample.0, 가중치 Dtype: torch.float32
모듈: layer2.1.conv1.0, 가중치 Dtype: torch.float32
모듈: layer2.1.conv2, 가중치 Dtype: torch.float32
모듈: layer3.0.conv1.0, 가중치 Dtype: torch.float32
모듈: layer3.0.conv2, 가중치 Dtype: torch.float32
모듈: layer3.0.downsample.0, 가중치 Dtype: torch.float32
모듈: layer3.1.conv1.0, 가중치 Dtype: torch.float32
모듈: layer3.1.conv2, 가중치 Dtype: torch.float32
모듈: layer4.0.conv1.0, 가중치 Dtype: torch.float32
모듈: layer4.0.conv2, 가중치 Dtype: torch.float32
모듈: layer4.0.downsample.0, 가중치 Dtype: torch.float32
모듈: layer4.1.conv1.0, 가중치 Dtype: torch.float32
모듈: layer4.1.conv2, 가중치 Dtype: torch.float32
모듈: fc, 가중치 Dtype: torch.float32


In [39]:
import torch
print(torch.__version__)

2.7.0+cu128
